In [108]:
import os
import scipy.io
import numpy as np

# 데이터 폴더 경로 설정
data_folder = '/Users/sihoon/Desktop/Eye_tracking'

mat_contents = scipy.io.loadmat('/Users/sihoon/Desktop/Eye_tracking/eye_data.mat')  # .mat 파일 불러오기

data = mat_contents['dpos'][:, [0, 1]]
label = mat_contents['dpos'][:, 2].round().astype(int)

# 데이터에서 인덱스 2670부터 2676까지의 행을 제거
data = np.delete(data, np.s_[2670:2677], axis=0)

# label 배열도 동일하게 처리
label = np.delete(label, np.s_[2670:2677], axis=0)

# label 배열에서 값이 4 이상인 인덱스를 찾습니다.
indices_to_replace = label >= 4
# 값이 4 이상인 요소를 0으로 치환합니다.
label[indices_to_replace] = 0

# label 배열에서 값이 3인 인덱스를 찾습니다.
replace = label == 3
# 값이 3인 요소를 2로 치환합니다.
label[replace] = 2

unique_labels = np.unique(label)
print(unique_labels)

print("불러온 데이터 배열 형태:", data.shape)

print("데이터의 차원 수:", data.ndim)

[0 1 2]
불러온 데이터 배열 형태: (2986, 2)
데이터의 차원 수: 2


In [109]:
# 데이터를 현재값 - 이전값으로 변환
data_diff = np.diff(data, axis=0)

label_cut = label[1:]

# 데이터와 레이블 확인
print("변환된 데이터 배열 형태:", data_diff.shape)
print("변환된 데이터 예시:")
print(data_diff[:5])
print("생성된 레이블 배열 형태:", label_cut.shape)
print("생성된 레이블 예시:")
print(label_cut[:5])

변환된 데이터 배열 형태: (2985, 2)
변환된 데이터 예시:
[[125.23475767  98.53576339]
 [-41.96555348 -33.74561077]
 [ 23.74113539  18.89643877]
 [-14.52814183 -11.57328158]
 [  9.27009183   7.77818888]]
생성된 레이블 배열 형태: (2985,)
생성된 레이블 예시:
[1 1 1 1 1]


In [110]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 준비
x = data_diff
y = label_cut

# 데이터 분할 (트레이닝 데이터셋과 테스트 데이터셋)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# LSTM 입력 데이터의 형태를 맞추기 위해 reshape
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

# 클래스 수 설정
num_classes = 3

# 모델 정의
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.1))
model.add(LSTM(32))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# 모델 훈련
history = model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=2)

# 모델 평가
accuracy = model.evaluate(x_test, y_test, verbose=0)[1]
print("Accuracy:", accuracy)

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_55 (LSTM)              (None, 1, 64)             17152     
                                                                 
 leaky_re_lu_52 (LeakyReLU)  (None, 1, 64)             0         
                                                                 
 lstm_56 (LSTM)              (None, 32)                12416     
                                                                 
 leaky_re_lu_53 (LeakyReLU)  (None, 32)                0         
                                                                 
 dense_51 (Dense)            (None, 32)                1056      
                                                                 
 dense_52 (Dense)            (None, 3)                 99        
                                                                 
Total params: 30723 (120.01 KB)
Trainable params: 307

In [95]:
# 테스트 데이터에 대한 예측
predicted_probabilities = model.predict(x_test)
predicted_classes = np.argmax(predicted_probabilities, axis=1)

# 예측된 클래스와 실제 레이블을 순서쌍으로 출력
print("Predicted | Actual")
print("-------------------")
for pred, actual in zip(predicted_classes, y_test):
    print(f"   {pred}      |    {actual}")


19/19 [==============================] - 0s 525us/step
Predicted | Actual
-------------------
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   2      |    2
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   2      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   2      |    2
   2      |    2
   1      |    1
   1      |    1
   2      |    2
   2      |    2
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   2      |    2
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    1
   1      |    2
   1      |    1
   1      |    1
   1      |    1
   2      |    2
   1      |    2
   1      |    1
   1 

In [96]:
# 일치하지 않는 예측 인덱스 찾기
incorrect_indices = np.where(predicted_classes != y_test)[0]

# 일치하지 않는 예측 출력
print("Incorrect Predictions:")
print("Predicted | Actual")
print("-------------------")
for index in incorrect_indices:
    pred = predicted_classes[index]
    actual = y_test[index]
    print(f"   {pred}      |    {actual}")


Incorrect Predictions:
Predicted | Actual
-------------------
   2      |    1
   1      |    2
   1      |    2
   1      |    2
   2      |    1
   1      |    2
   2      |    1
   2      |    1
   2      |    1
   1      |    2
   1      |    2
   1      |    2
   2      |    0
   1      |    2
   2      |    1
   2      |    1
   1      |    2
   2      |    0
   2      |    1
   2      |    1
   1      |    2


클래스 4 : 0.9597 // 최적화 : rmsprop
클래스 3 : 0.9631490707397461 // 최적화 : rmsprop, adam // leaky relu 추가.
클래스 4 : 0.9396985173225403 // 최적화 : rmsprop // leaky relu 추가.
클래스 3 : 0.9564489126205444 // 최적화 : sgd // leaky relu 추가.
클래스 4 : 0.9463986754417419 // 최적화 : sgd // leaky relu 추가.
클래스 3 : 0.9698492288589478 // 최적화 : adam // leaky relu 추가. // dropout 0.2 -> 0.1
클래스 3 : 0.9648241400718689 // 최적화 : adam // leaky relu 추가. // dropout x





